In [1]:
import numpy as np
from uncertainties import ufloat
from uncertainties import unumpy

import astropy.io.fits as pyfits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time
from astropy.wcs import WCS

from sys import argv
import warnings
import time
import os

In [2]:
path_log_imfit   = '../File_Log_imfit/'
path_log_imstat  = '../File_Log_imstat/'
path_data        = '../Data_NoSelfcal/Data_NoSelfCal_Rg/'

# Observational bands
Band_arr     = np.array(['Ka', 'Ka', 'Q', 'B6', 'B6', 'K', 'B7', 'K', 'Ka', 'B7', 'B6', 'B7', 'B3', 'B6', 'B6', 'B4', 'B4'])

outnames_log = [
                'iras4a2_2013_10_21_Ka_Tobin',
                'iras4a2_2014_02_24_Ka_Tobin',
                'iras4a2_2014_10_13_Q_Liu',
                'iras4a2_2015_06_13_B6_Sakai',
                'iras4a2_2015_09_27_B6_Tobin',
                'iras4a2_2016_06_04_K_Liu',
                'iras4a2_2016_07_23_B7_Su',
                'iras4a2_2016_08_06_K_Liu',
                'iras4a2_2016_09_04_Ka_Liu',
                'iras4a2_2016_09_06_B7_Lai',
                'iras4a2_2016_11_04_B6_Tao',
                'iras4a2_2016_12_14_B7_Su',
                'iras4a2_2017_08_17_B3_Cox',
                'iras4a2_2017_08_28_B6_Maury',
                'iras4a2_2017_12_17_B6_Tobin',
                'iras4a2_2018_10_16_B4_Francesco',
                'iras4a2_2019_07_20_B4_Lai'
                ]

outnames_fits = [
                'n1333iras4a_2013_10_21_Ka_Tobin',
                'n1333iras4a_2014_02_24_Ka_Tobin',
                'n1333iras4a_2014_10_13_Q_Liu',
                'n1333iras4a_2015_06_13_B6_Sakai',
                'n1333iras4a_2015_09_27_B6_Tobin',
                'n1333iras4a_2016_06_04_K_Liu',
                'n1333iras4a_2016_07_23_B7_Su',
                'n1333iras4a_2016_08_06_K_Liu',
                'n1333iras4a_2016_09_04_Ka_Liu',
                'n1333iras4a_2016_09_06_B7_Lai',
                'n1333iras4a_2016_11_04_B6_Tao',
                'n1333iras4a_2016_12_14_B7_Su',
                'n1333iras4a_2017_08_17_B3_Cox',
                'n1333iras4a_2017_08_28_B6_Maury',
                'n1333iras4a_2017_12_17_B6_Tobin',
                'n1333iras4a_2018_10_16_B4_Francesco',
                'n1333iras4a_2019_07_20_B4_Lai'
                ]

Coord_err_4A2_per = [
                    10e-2, # 00: Ka, self-cal.           , not fit
                    10e-2, # 01: Ka, self-cal.           , not fit
                    10e-2, # 02: Q,                      , not fit
                    5e-2,  # 03: B6, Cy1 --> 50 mas      , not fit
                    5e-2,  # 04: B6, Cy1 --> 50 mas      , not fit
                    10e-2, # 05: K
                    5e-2,  # 06: B7, Cy3 --> 50 mas
                    10e-2, # 07: K
                    10e-2, # 08: Ka
                    5e-2,  # 09: B7, Cy3 --> 50 mas
                    5e-2,  # 10: B6, Cy4                 , not fit
                    5e-2,  # 11: B7, Cy3 --> 50 mas      , not fit
                    5e-2,  # 12: B3, Cy4
                    5e-2,  # 13: B6, Cy4
                    5e-2,  # 14: B6, Cy5
                    5e-2,  # 15: B4, Cy6                 , not fit
                    5e-2   # 16: B4, Cy6
                    ]
Coord_err_4A2_50mas = [3, 4, 6, 9, 11]         
drop_list           = [0, 1, 2, 3, 4, 10, 11, 15] 

### Positions of NGC1333 IRAS4A2

Hint: position inforamation is obtained from CASA imfit, run the following scripts first. 
- script_iras4a_rg2b4.py
- script_iras4a_imfit.py
- script_iras4a_imstat.py

### (1) From the 2D gaussian fitting (CASA imfit task)

In [3]:
# Obtain the coordinate information (Coord_4A2_arr)

ra_gaufit_hms_list, ra_err_gaufit_sec_list, ra_err_gaufit_arcs_list = [], [], []
dec_gaufit_dms_list, dec_err_gaufit_arcs_list = [], []
Coord_4A2_list = []

for i, outname_log in enumerate(outnames_log):

    # Set the log filename
    logName  = '%s%s.log'%(path_log_imfit, outname_log)

    # read the log file
    file  = open(logName,"r+") 
    lines = file.read().splitlines()
    file.close()
    
    # save the ra & dec
    ra_h        = lines[20].split()[2].split(':')[0]
    ra_m        = lines[20].split()[2].split(':')[1]
    ra_s        = lines[20].split()[2].split(':')[2]   
    ra_hms      = ra_h + 'h' + ra_m + 'm' + ra_s + 's'
    ra_err_sec  = lines[20].split()[4]
    ra_err_arcs = lines[20].split()[6].split('(')[1]
    
    dec_d       = lines[21].split()[2].split('.', 2)[0]
    dec_m       = lines[21].split()[2].split('.', 2)[1]
    dec_s       = lines[21].split()[2].split('.', 2)[2]
    dec_dms     = dec_d + 'd' + dec_m + 'm' + dec_s + 's'
    dec_err_arcs = lines[21].split()[4]
    
    Coord_4A2 = SkyCoord(ra_hms, dec_dms, frame='icrs')
    
    # add to the list
    ra_gaufit_hms_list.append(ra_hms)
    ra_err_gaufit_sec_list.append(ra_err_sec)
    ra_err_gaufit_arcs_list.append(ra_err_arcs)
    dec_gaufit_dms_list.append(dec_dms)
    dec_err_gaufit_arcs_list.append(dec_err_arcs)
    Coord_4A2_list.append(Coord_4A2) 
     
# convert list to array
ra_gaufit_hms_arr       = np.array(ra_gaufit_hms_list)
ra_err_gaufit_sec_arr   = np.array(ra_err_gaufit_sec_list).astype(np.float)
ra_err_gaufit_arcs_arr  = np.array(ra_err_gaufit_arcs_list).astype(np.float)
dec_gaufit_dms_arr      = np.array(dec_gaufit_dms_list)
dec_err_gaufit_arcs_arr = np.array(dec_err_gaufit_arcs_list).astype(np.float)
Coord_4A2_arr           = np.array(Coord_4A2_list)

In [4]:
# Coordinate of IRAS4A2 [RA:deg; Dec:deg]
# hints: Gaussian fitting via CASA imfit
# print (Coord_4A2_arr, '\n')
print (ra_gaufit_hms_arr, '\n')
print (dec_gaufit_dms_arr, '\n')

['03h29m10.426788s' '03h29m10.426359s' '03h29m10.42960s'
 '03h29m10.431467s' '03h29m10.42496s' '03h29m10.427422s'
 '03h29m10.429116s' '03h29m10.428520s' '03h29m10.428368s'
 '03h29m10.430085s' '03h29m10.429762s' '03h29m10.431893s'
 '03h29m10.429977s' '03h29m10.430320s' '03h29m10.429432s'
 '03h29m10.43160s' '03h29m10.430836s'] 

['+031d13m32.114935s' '+031d13m32.103306s' '+031d13m32.15433s'
 '+031d13m32.116936s' '+031d13m32.17266s' '+031d13m32.093957s'
 '+031d13m32.060897s' '+031d13m32.081908s' '+031d13m32.082498s'
 '+031d13m32.059731s' '+031d13m32.068950s' '+031d13m32.101404s'
 '+031d13m32.062794s' '+031d13m32.059614s' '+031d13m32.060100s'
 '+031d13m32.08892s' '+031d13m32.035269s'] 



### (2) From the maximum pixel (CASA imstat task)

In [5]:
# Obtain the coordinate information (Coord_4A2_arr)

ra_maxpx_hms_list, dec_maxpx_dms_list = [], []
Coord_4A2_maxpx_list = []

for i, outname_log in enumerate(outnames_log):
 
    # Set the log filename
    logName  = '%s%s_imstat.log'%(path_log_imstat, outname_log)

    # read the log file
    file  = open(logName,"r+") 
    lines = file.read().splitlines()
    file.close()
    
    
    # save the ra & dec
    ra_h        = lines[13].split()[8].split(',')[0].split(':')[0]
    ra_m        = lines[13].split()[8].split(',')[0].split(':')[1]
    ra_s        = lines[13].split()[8].split(',')[0].split(':')[2]
    ra_hms      = ra_h + 'h' + ra_m + 'm' + ra_s + 's'
    
    dec_d       = lines[13].split()[9].split(',')[0].split('.', 2)[0]
    dec_m       = lines[13].split()[9].split(',')[0].split('.', 2)[1]
    dec_s       = lines[13].split()[9].split(',')[0].split('.', 2)[2]
    dec_dms     = dec_d + 'd' + dec_m + 'm' + dec_s + 's'
    
    Coord_4A2_maxpx = SkyCoord(ra_hms, dec_dms, frame='icrs')
    # maxpx_value     = lines[9].split()[5]
    
    # add to the list
    ra_maxpx_hms_list.append(ra_hms)
    dec_maxpx_dms_list.append(dec_dms)
    Coord_4A2_maxpx_list.append(Coord_4A2_maxpx) 
     
# convert list to array
ra_maxpx_hms_arr       = np.array(ra_maxpx_hms_list)
dec_maxpx_dms_arr      = np.array(dec_maxpx_dms_list)
Coord_4A2_maxpx_arr    = np.array(Coord_4A2_maxpx_list)

In [6]:
# Coordinate of IRAS4A2 [RA:deg; Dec:deg]
# hints: Maxmum pixel via CASA imstat
# print (Coord_4A2_maxpx_arr, '\n')
print (ra_maxpx_hms_arr, '\n')
print (dec_maxpx_dms_arr, '\n')

['03h29m10.4267s' '03h29m10.4267s' '03h29m10.4298s' '03h29m10.4322s'
 '03h29m10.4244s' '03h29m10.4275s' '03h29m10.4283s' '03h29m10.4283s'
 '03h29m10.4283s' '03h29m10.4298s' '03h29m10.4298s' '03h29m10.4306s'
 '03h29m10.4298s' '03h29m10.4306s' '03h29m10.4290s' '03h29m10.4306s'
 '03h29m10.4306s'] 

['+31d13m32.1198s' '+31d13m32.0998s' '+31d13m32.1598s' '+31d13m32.1198s'
 '+31d13m32.1798s' '+31d13m32.0898s' '+31d13m32.0598s' '+31d13m32.0798s'
 '+31d13m32.0798s' '+31d13m32.0598s' '+31d13m32.0598s' '+31d13m32.1198s'
 '+31d13m32.0598s' '+31d13m32.0598s' '+31d13m32.0698s' '+31d13m32.0998s'
 '+31d13m32.0398s'] 



### Observation time & position uncertainties from the header 

In [7]:
# obtain the time information (Time_arr)

Time_list = []
Coord_err_4A2_list = []

for i, outname_fits in enumerate(outnames_fits):
    
    # Get the FITS and header
    DataName  = '%s%s_I.image.fits'%(path_data, outname_fits)
    I_hd      = pyfits.getheader(DataName)
    
    # Get the frequency, beam, and observation time information from header
    hd_freq = float(I_hd['RESTFRQ'])  # [Hz]
    hd_bmaj = float(I_hd['BMAJ'])     # [deg]
    hd_bmin = float(I_hd['BMIN'])     # [deg]
    hd_bpa  = float(I_hd['BPA'])      # [deg]
    hd_time = I_hd['DATE-OBS']
    
    # Observation time
    #Time_ymd = Time(hd_time.split('T')[0])
    Time_ymdhms = Time(hd_time, format='isot', scale='utc')
    Time_list.append(Time_ymdhms)
    
    # Uncertainties of coordinate towards IRAS4A2 [arcsec]
    beam_maj_arcs     = hd_bmaj*3600
    Coord_err_4A2     = beam_maj_arcs*Coord_err_4A2_per[i] if i not in Coord_err_4A2_50mas else 0.05
    Coord_err_4A2     = float('%.3f' % Coord_err_4A2)
    Coord_err_4A2_list.append(Coord_err_4A2)
    
# convert list to array    
Time_arr          = np.array(Time_list)
Coord_err_4A2_arr = np.array(Coord_err_4A2_list)

In [8]:
print (Time_arr)

[<Time object: scale='utc' format='isot' value=2013-10-21T09:35:52.500>
 <Time object: scale='utc' format='isot' value=2014-02-24T03:05:43.500>
 <Time object: scale='utc' format='isot' value=2014-10-13T11:03:22.500>
 <Time object: scale='utc' format='isot' value=2015-06-13T15:41:32.688>
 <Time object: scale='utc' format='isot' value=2015-09-27T06:22:04.320>
 <Time object: scale='utc' format='isot' value=2016-06-04T14:00:13.500>
 <Time object: scale='utc' format='isot' value=2016-07-23T13:09:39.264>
 <Time object: scale='utc' format='isot' value=2016-08-06T14:57:10.500>
 <Time object: scale='utc' format='isot' value=2016-09-04T14:37:13.500>
 <Time object: scale='utc' format='isot' value=2016-09-06T07:54:16.512>
 <Time object: scale='utc' format='isot' value=2016-11-04T03:56:27.360>
 <Time object: scale='utc' format='isot' value=2016-12-14T02:46:23.664>
 <Time object: scale='utc' format='isot' value=2017-08-17T09:22:12.456>
 <Time object: scale='utc' format='isot' value=2017-08-28T10:12:

In [9]:
print (Coord_err_4A2_arr)

[0.017 0.006 0.039 0.05  0.05  0.025 0.05  0.024 0.02  0.05  0.019 0.05
 0.013 0.008 0.014 0.017 0.004]


### Save the information to the textfile

In [10]:
%%time

txt_table_lists,  txt_lists= [], []
for i, elm in enumerate(Band_arr):
    
    # Time
    Time_ymdhms    = Time_arr[i].iso                             # Observationl Time [y-m-d h:m:s]
    Time_ymdThms   = Time_arr[i].isot                            # Observationl Time [y-m-dTh:m:s]
    Time_ymd       = Time_ymdhms.split()[0]                      # Observationl Time [y-m-d]
    Time_yr        = Time_arr[i].decimalyear                     # Observationl Time [year]
    Time_jd        = Time_arr[i].jd                              # Observationl Time [Julian Day]
    
    # Coordinate System (1) imfit
    radec_hmsdms   = Coord_4A2_arr[i].to_string('hmsdms')        # RA & Dec [hmsdms]
    radec_err_arcs = Coord_err_4A2_arr[i]                        # Uncertainties of RA & Dec [arcsec]
    radec_err_deg  = np.divide(Coord_err_4A2_arr[i], 3600)       # Uncertainties of RA & Dec [degree]
    radec_dig_deg  = int((np.floor(np.log10(radec_err_deg))*-1)) # Significant digit of RA & Dec
    
    ra_hms         = str(radec_hmsdms).split()[0]                # Right Ascension [hms]
    dec_dms        = str(radec_hmsdms).split()[1]                # Declination [dms]
    ra_deg         = Coord_4A2_arr[i].ra.deg                     # Right Ascension [degree]
    dec_deg        = Coord_4A2_arr[i].dec.deg                    # Declination [degree]
    ra_dig_deg     = round(ra_deg, radec_dig_deg+1)              # Right Ascension with significant digit [degree]
    dec_dig_deg    = round(dec_deg, radec_dig_deg+1)             # Declination with significant digit  [degree]

    # Coordinate System (2) imstat
    radec_maxpx_hmsdms   = Coord_4A2_maxpx_arr[i].to_string('hmsdms')        # RA & Dec [hmsdms]
    radec_maxpx_dig_deg  = int((np.floor(np.log10(radec_err_deg))*-1))       # Significant digit of RA & Dec
    
    ra_maxpx_hms         = str(radec_maxpx_hmsdms).split()[0]                # Right Ascension [hms]
    dec_maxpx_dms        = str(radec_maxpx_hmsdms).split()[1]                # Declination [dms]
    ra_maxpx_deg         = Coord_4A2_maxpx_arr[i].ra.deg                     # Right Ascension [degree]
    dec_maxpx_deg        = Coord_4A2_maxpx_arr[i].dec.deg                    # Declination [degree]
    ra_maxpx_dig_deg     = round(ra_maxpx_deg, radec_maxpx_dig_deg+1)        # Right Ascension with significant digit [degree]
    dec_maxpx_dig_deg    = round(dec_maxpx_deg, radec_maxpx_dig_deg+1)       # Declination with significant digit  [degree]


    

    # (1) the table for putting on the paper
    # save the txt file   
    txt_table_list = [i+1, elm, Time_ymdhms, Time_yr, Time_jd, \
                      ra_hms, dec_dms, ra_dig_deg, dec_dig_deg, radec_err_deg, \
                      ra_maxpx_hms, dec_maxpx_dms, ra_maxpx_dig_deg, dec_maxpx_dig_deg, '\\\\']
    # print (txt_list)
    txt_table_lists.append(txt_table_list)
    np.savetxt('Position_4A2_all_table.txt', txt_table_lists, fmt='%s', delimiter=' & ')
    
    
    # (2) table for data analysis
    # save the txt file   
    txt_list = [i+1, elm, Time_ymdThms, Time_ymd, Time_yr, Time_jd, \
                ra_hms, dec_dms, ra_dig_deg, dec_dig_deg, ra_deg, dec_deg, radec_err_deg, \
                ra_maxpx_hms, dec_maxpx_dms, ra_maxpx_dig_deg, dec_maxpx_dig_deg, ra_maxpx_deg, dec_maxpx_deg]
    # print (txt_list)
    txt_lists.append(txt_list)
        
    np.savetxt('Position_4A2_all.txt', txt_lists, fmt='%s')

CPU times: user 71.9 ms, sys: 13 ms, total: 84.9 ms
Wall time: 121 ms
